# 1. 비즈니스 뉴스 & 일반 뉴스 합치기

In [19]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

In [20]:
# news_df1 = pd.read_excel('/Users/parkjunhyeong/Desktop/nytimes_ver1.1.xlsx', sheet_name = 'Sheet1', index_col = 0)
# news_df2 = pd.read_excel('/Users/parkjunhyeong/Desktop/nytimes2_ver0.1.xlsx', sheet_name = 'Sheet1', index_col = 0)

In [21]:
# news_df1.columns

In [22]:
# news_df2.rename(columns = {'hrefs' : 'URL', 'publish_date' : 'Publish Date', 'title' : 'Title', 'main_text' : 'Main Text',
#                    'len' : 'Length'}, inplace = True ) 
# news_df2.columns

In [23]:
# news_df1 = news_df1[['URL', 'Publish Date', 'Title', 'Main Text', 'Ticker', 'Description', 'Name', 'Length']]
# news_df2 = news_df2[['URL', 'Publish Date', 'Title', 'Main Text', 'Ticker', 'Description', 'Name', 'Length']]

In [24]:
# news_df = pd.concat([news_df1, news_df2])
# news_df

In [25]:
# news_df = news_df[news_df['Length'] == 1]
# news_df

In [26]:
news_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/뉴스 라벨링 취합_ver0.2.xlsx', sheet_name = 'Sheet3')
news_df['Ticker'] = news_df['Ticker'].apply(lambda x : eval(x))
news_df['Description'] = news_df['Description'].apply(lambda x : eval(x))

In [27]:
for i in range(len(news_df)):
    if len(news_df['Ticker'][i]) > 1:
        news_df['Ticker'][i] = [news_df['Ticker'][i][0]]
        news_df['Description'][i] = [news_df['Description'][i][0]]

/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5211/2733437392.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['Ticker'][i] = [news_df['Ticker'][i][0]]
/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5211/2733437392.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['Description'][i] = [news_df['Description'][i][0]]


In [28]:
#제대로 되었는지 확인하기
for i in range(len(news_df)):
    if len(news_df['Ticker'][i]) != 1:
        print(i)

# 2. 기업별 상장일 이전에 나온 뉴스 삭제

In [29]:
stock_path = '/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/04. 주가 데이터/01. Yahoo Finance'
companies = os.listdir('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/04. 주가 데이터/01. Yahoo Finance')


In [30]:
len(companies)

220

In [31]:
news_df['Date Comparison'] = True

for i in tqdm(range(len(news_df))):
    #뉴스 데이터에서 필요 정보 가져오기
    ticker = news_df.loc[i, 'Ticker'][0]
    publish_date = str(news_df.loc[i, 'Publish Date'])
    publish_date = datetime(int(publish_date[:4]), int(publish_date[5:7]), int(publish_date[8:10]))
    #주가 데이터에서 필요 정보 가져오기
    stock_df = pd.read_csv(stock_path + '/' + ticker + '.csv')
    start_date = stock_df.iloc[0,0]
    start_date = datetime(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:10]))
    
    if start_date >= publish_date:
        news_df.iloc[i, -1] = False
    else: 
        pass

100%|██████████████████████████████████████| 4802/4802 [00:06<00:00, 793.17it/s]


In [32]:
news_df['Date Comparison'].value_counts()

True     4561
False     241
Name: Date Comparison, dtype: int64

In [33]:
news_df = news_df[news_df['Date Comparison'] == True]
news_df.reset_index(drop = True, inplace = True)

# 3. 날짜별 Stock Price Excess Return 계산하기

In [34]:
Market_df = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/01. 메타 데이터/F-F_Research_Data_Factors_daily.CSV')
Market_df.dropna(inplace = True)
Market_df['Date'] = Market_df['Date'].apply(lambda x : x[:4] + '-' + x[4:6] + '-' + x[6:])

In [35]:
news_df['Investor Reaction'] = 0

/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5211/3172078703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['Investor Reaction'] = 0


In [ ]:
for i in tqdm(range(len(news_df))):
    try:
        if news_df.iloc[i, -2] == False:
            pass
        else:
            ticker = news_df.loc[i, 'Ticker'][0]
            publish_date = news_df.loc[i, 'Publish Date']
            publish_date = str(publish_date)[:-9]
            #주가 데이터에서 필요 정보 가져오기
            stock_df = pd.read_csv(stock_path + '/' + ticker + '.csv')
            #news publish date와 일치하는 영업일이 있는 경우
            if publish_date in list(stock_df['Date']):
                ER_total = 0
                for j in range(5):
                    #Return 계산
                    index1 = stock_df[stock_df['Date'] == publish_date].index
                    Return = (float(stock_df.loc[index1 + j, 'Adj Close']) - float(stock_df.loc[index1 + j - 1, 'Adj Close'])) / float(stock_df.loc[index1 + j - 1, 'Adj Close'])
                    #Market Rate of Return 계산
                    index2 = Market_df[Market_df['Date'] == publish_date].index
                    Market_Return = (float(Market_df.loc[index2 + j, 'Mkt-RF']) + float(Market_df.loc[index2 + j, 'RF'])) / 100
                    #Excess Return 계산
                    #ER = math.log10(1 + (Return - Market_Return) / 10)
                    ER = math.log10(1 + (Return - Market_Return))
                    ER_total += ER
                news_df.iloc[i,-1] = ER_total
            #news publish date와 일치하는 영업일이 없는 경우
            else:
                publish_datetime = datetime(int(publish_date[:4]), int(publish_date[5:7]), int(publish_date[8:10]))
                for k in range(10):
                    if (publish_datetime + timedelta(days = k)).strftime("%Y-%m-%d") in list(stock_df['Date']):
                        publish_date = (publish_datetime + timedelta(days = k)).strftime("%Y-%m-%d")
                        break
                ER_total = 0
                for j in range(5):
                    #Return 계산
                    index1 = stock_df[stock_df['Date'] == publish_date].index
                    Return = (float(stock_df.loc[index1 + j, 'Adj Close']) - float(stock_df.loc[index1 + j - 1, 'Adj Close'])) / float(stock_df.loc[index1 + j - 1, 'Adj Close'])
                    #Market Rate of Return 계산
                    index2 = Market_df[Market_df['Date'] == publish_date].index
                    Market_Return = (float(Market_df.loc[index2 + j, 'Mkt-RF']) + float(Market_df.loc[index2 + j, 'RF'])) / 100
                    #Excess Return 계산
                    #ER = math.log10(1 + (Return - Market_Return) / 10)
                    ER = math.log10(1 + (Return - Market_Return))
                    ER_total += ER
                news_df.iloc[i,-1] = ER_total
                
    except Exception as e :
        print(ticker, publish_date)
        print(e)

  0%|                                                  | 0/4561 [00:00<?, ?it/s]/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5211/2597140441.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.iloc[i,-1] = ER_total
 35%|█████████████▎                        | 1605/4561 [00:16<00:28, 102.87it/s]

In [39]:
news_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction
0,https://www.nytimes.com/2019/01/01/business/me...,2019-01-01,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show “Pa...,[NFLX],[Netflix Inc],ESG,Negative,0.064605
1,https://www.nytimes.com/2019/01/02/business/ju...,2019-01-02,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,[ACN],[Accenture Plc Class A],Non-ESG,Positive,-0.001291
2,https://www.nytimes.com/2019/01/02/business/te...,2019-01-02,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,[TSLA],[Tesla Inc],Financial Performance,Negative,-0.010247
3,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,[BMY],[Bristol Myers Squibb],Financial Performance,Neutral,-0.056534
4,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Was Apple’s Warning a Surprise? Not to Its Sup...,"Apple’s stock fell 10 percent on Thursday, its...",[AAPL],[Apple Inc],Financial Performance,Negative,-0.027306
...,...,...,...,...,...,...,...,...,...
4797,https://www.nytimes.com/2022/10/18/technology/...,2022-10-18,Amazon Labor Union Loses Election at Warehouse...,Mr. Smalls said the election “wasn’t free and ...,[AMZN],[Amazon.Com Inc.],ESG,Positive,0.010129
4798,https://www.nytimes.com/2022/10/20/technology/...,2022-10-20,Texas Sues Google for Collecting Biometric Dat...,"José Castañeda, a Google spokesman, said in a ...",[GOOG],[Alphabet Inc Class C],ESG,Negative,-0.040082
4799,https://www.nytimes.com/2022/10/25/technology/...,2022-10-25,Microsoft Reports Slowest Revenue Growth in Fi...,"In the most recent quarter, softness in the gl...",[MSFT],[Microsoft Corp],Financial Performance,Positive,-0.036752
4800,https://www.nytimes.com/2022/10/27/technology/...,2022-10-27,"Apple’s iPhone Powers Growth, but Signs Point ...",When Apple released the 16th version of its iP...,[AAPL],[Apple Inc],Financial Performance,Positive,-0.003742


In [40]:
news_df.to_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/Composed_Data_ver1.1.csv')